# collect and count adjacent words
given a list of keywords, this routine considers each word in turn, crawls through the corpus and finds all adjacent words (of the chosen type)  to that word. adjacent word means the word immediately preceding or following the target word, in the same sentence as the target word.

for each keyword, it produces a data frame of adjacent words, whether the adjacent word preceeded or followed, and the frequency of such pairing. 

In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [2]:
# input
data_dir = Path("/Users/oholm/annad/ordtidni/Gigaword/output/")
results_dir = Path("/Users/oholm/annad/ordtidni/data/adjacent_adjectives")
input_file_path = data_dir.glob('*/**/*')
filepaths = [item for item in input_file_path if (item.is_file() and str(item).endswith('.parquet'))]
# input: nouns as keywords frequencies
noun_freq_filepath = data_dir/'word_occurrence_counts.parquet'
# output
noun_adj_filepath = data_dir/'noun_adj/nouns_and_pre_post_adj_freq.parquet'

In [3]:
noun_frequencies =\
    pd.read_parquet(noun_freq_filepath)\
    .astype(int)\
    .sort_values(by='occurance_count', ascending=False)

noun_frequencies.head(10)

,occurance_count
ár,5743010
mál,3226065
maður,3118502
dagur,2824253
land,1972793
lag,1967438
leikur,1812649
Ísland,1758909
vegur,1701757
tími,1554961


In [4]:
# iterate for each keyword in the list
for keyword in noun_frequencies.index[:3]:
    print(keyword)
keyword = noun_frequencies.index[0]

ár
mál
maður


In [5]:
keywords =noun_frequencies.index[0:30]
print(keywords)

Index(['ár', 'mál', 'maður', 'dagur', 'land', 'lag', 'leikur', 'Ísland',
       'vegur', 'tími', 'lið', 'fólk', 'staður', 'króna', 'forseti', 'barn',
       'félag', 'fyrirtæki', 'stefnandi', 'mark', 'kona', 'leið', 'Reykjavík',
       'hönd', 'starf', 'ríki', 'mánuður', 'hluti', 'milljón', 'þáttur'],
      dtype='object')


In [6]:
%%time
print('crawling through corpus. processing', len(filepaths),\
          'corpus files', len(keywords), ' times')

for keyword in tqdm(keywords):
    
    all_matches = pd.DataFrame()
    print('----'*9, keyword, '----'*9)
    for filepath in tqdm(filepaths):
#         print(filepath.parent)
        corpus_doc = pd.read_parquet(filepath)
        # mark up corpus terms by properties of the previous word
        corpus_doc['preceding_same_sentence'] =\
            ((corpus_doc['Sentence'].shift(-1) == corpus_doc['Sentence'])\
            & (corpus_doc['Paragraph'].shift(-1) == corpus_doc['Paragraph']))
        corpus_doc['preceding_lemma'] = corpus_doc['Lemma'].shift(-1)
        corpus_doc['preceding_isadj'] = corpus_doc['POS'].shift(-1).str.startswith('l')
        # mark up corpus terms by properties of the following word
        corpus_doc['following_same_sentence'] =\
            ((corpus_doc['Sentence'].shift(1) == corpus_doc['Sentence'])\
            & (corpus_doc['Paragraph'].shift(1) == corpus_doc['Paragraph']))
        corpus_doc['following_lemma'] = corpus_doc['Lemma'].shift(1)
        corpus_doc['following_isadj'] = corpus_doc['POS'].shift(1).str.startswith('l')
#         display(corpus_doc.head(100).tail(10)) 

        # focus on nouns that are preceded by or followed by an adjective.
        # 1. first filter on targer keyword.
        # 2. then ensure preceding word is in same sentence
        # 3. then ensure preceding word is adjective
        # (repeat for following word)
        targets_with_matching_preceding =\
            (corpus_doc['Lemma']==keyword)\
            & (corpus_doc['preceding_same_sentence'] )\
            & (corpus_doc['preceding_isadj'])
        targets_with_matching_following =\
            (corpus_doc['Lemma']==keyword)\
            & (corpus_doc['following_same_sentence'] )\
            & (corpus_doc['following_isadj'])
        # collect matching preceding lemmas only
        target_preceded_occurences_df =\
            corpus_doc[['preceding_lemma']][targets_with_matching_preceding]
        target_preceded_occurences_df['placement'] = 'preceding'
        target_preceded_occurences_df\
            .rename(columns={'preceding_lemma':'adjacent_lemma'},
                    inplace=True)
#         display(target_preceded_occurences_df.head(3))
        # collect matching following lemmas only
        target_followed_occurences_df =\
            corpus_doc[['following_lemma']][targets_with_matching_following]
        target_followed_occurences_df['placement'] = 'following'
        target_followed_occurences_df\
            .rename(columns={'following_lemma':'adjacent_lemma'},
                    inplace=True)
        # combine all found so far:
        all_matches =\
            pd.concat([all_matches,
                       target_preceded_occurences_df,
                       target_followed_occurences_df], 
                      axis=0)
        
    all_matches['target_lemma'] = keyword
    display(all_matches)
    print('found', all_matches.shape[0], 'matching pairs')
    output_file = results_dir/ (keyword+'_adjacent_adjectives.parquet')
    all_matches.to_parquet(output_file)




  0%|          | 0/661 [00:00<?, ?it/s]

crawling through corpus. processing 661 corpus files 30  times
------------------------------------ ár ------------------------------------



100%|██████████| 661/661 [6:33:32<00:00, 35.72s/it]


,adjacent_lemma,placement,target_lemma
377,nýr,preceding,ár
1219,undanfarinn,preceding,ár
1318,undanfarinn,preceding,ár
6520,síðari,preceding,ár
8370,rúmur,preceding,ár
...,...,...,...
51522,efnahagslegur,following,ár
52807,gamall,following,ár
53750,gamall,following,ár
58606,liðinn,following,ár


found 1403097 matching pairs


...

found 434269 matching pairs


  0%|          | 0/661 [00:00<?, ?it/s]

------------------------------------ maður ------------------------------------



 86%|████████▌ | 566/661 [46:18<1:00:15, 38.06s/it]

KeyboardInterrupt: 

In [ ]:
# 

In [44]:
# import datetime as DT
# d = {'case'        : pd.Series([1,1,1,1,2]),
#      'open'        : pd.Series([DT.datetime(2014, 3, 2), DT.datetime(2014, 3, 2),DT.datetime(2014, 3, 2),DT.datetime(2014, 3, 2),DT.datetime(2014, 3, 2)]),
#      'change'      : pd.Series([DT.datetime(2014, 3, 8), DT.datetime(2014, 4, 8),DT.datetime(2014, 5, 8),DT.datetime(2014, 6, 8),DT.datetime(2014, 6, 8)]),
#      'StartEvent'  : pd.Series(['Homeless','Homeless','Homeless','Homeless','Jail']),
#      'ChangeEvent' : pd.Series(['Homeless','irrelevant','Homeless','Jail','Jail']),
#      'close'       : pd.Series([DT.datetime(2015, 3, 2), DT.datetime(2015, 3, 2),DT.datetime(2015, 3, 2),DT.datetime(2015, 3, 2),DT.datetime(2015, 3, 2)])
#     }
# df=pd.DataFrame(d)
# df2 = df
# df['ok'] = True
# df2['notok'] = False
# pd.concat([df,df2],axis=0)

,case,open,change,StartEvent,ChangeEvent,close,ok,notok
0,1,2014-03-02,2014-03-08,Homeless,Homeless,2015-03-02,True,False
1,1,2014-03-02,2014-04-08,Homeless,irrelevant,2015-03-02,True,False
2,1,2014-03-02,2014-05-08,Homeless,Homeless,2015-03-02,True,False
3,1,2014-03-02,2014-06-08,Homeless,Jail,2015-03-02,True,False
4,2,2014-03-02,2014-06-08,Jail,Jail,2015-03-02,True,False
0,1,2014-03-02,2014-03-08,Homeless,Homeless,2015-03-02,True,False
1,1,2014-03-02,2014-04-08,Homeless,irrelevant,2015-03-02,True,False
2,1,2014-03-02,2014-05-08,Homeless,Homeless,2015-03-02,True,False
3,1,2014-03-02,2014-06-08,Homeless,Jail,2015-03-02,True,False
4,2,2014-03-02,2014-06-08,Jail,Jail,2015-03-02,True,False


In [ ]:
# def get_nearest_words(corpus_df, lemma_to_look_for, pos_class):
#     """
#     given a lemma, and a corpus dataframe, find all instances of the word in the dataframe, as well as the words
#     preceeding and following each. 
#     and if that word is an adjective, update the count 
#     """
    
    
